# PyCPT Version 2.2dev

This is an example of a PyCPT Version 2 seasonal climate forecasting workflow. This notebook can be adapted to suit your exact needs through modifications of the code. This notebook uses PyCPT v2 utilities to 

1. download data from the IRI Data Library (through the CPT-DL python library) 
2. Run bias-correction using the IRI Climate Predictability Tool (through its companion python library, CPT-CORE) 
3. Plot skills scores and spatial loadings
4. Produce a multi-model ensemble forecast by taking the simple average of the bias-corrected members
5. Plots skill scores, deterministic forecasts, probabilistic forecasts, and exceedance probabilities for this NextGen MME forecast. 

PyCPT Version 2 was primarily designed and implemented by Kyle Hall

#### Imports - This cell imports PyCPTv2 libraries 

In [ ]:
import cptdl as dl 
import cptcore as cc 
import cptextras as ce
import datetime as dt
import numpy as np
from pathlib import Path
from scipy.stats import norm, t
import xarray as xr 

import notebook

#### Define Case Directory

In [ ]:
caseDir = "pycpt_WAfricaJAS_startJun"  

#### Parameters - This cell defines the parameters of your analysis

In [ ]:
MOS = 'CCA' # must be one of 'CCA', 'PCR', or "None"
predictor_names = [ "SPEAR.PRCP", "CCSM4.PRCP", "CanSIPSIC3.PRCP","CFSv2.PRCP","GEOSS2S.PRCP"]# ['CFSv2.PRCP','SEAS5.PRCP']
predictand_name = 'UCSB.PRCP'

#predictor_names = ['SPEAR.TMAX','CanSIPSIC3.TMAX']
#predictand_name = 'UCSB.TMAX'

# use dl.observations.keys() to see all options for predictand 
# and dl.hindcasts.keys() to see all options for predictors
# make sure your first_year & final_year are compatible with 
# your selections for your predictors and predictands 

download_args = { 
   # 'fdate':
   #   the initialization date of the model forecasts / hindcasts
   #   this field is defined by a python datetime.datetime object
   #   for example: dt.datetime(2022, 5, 1) # YYYY, MM, DD as integers
   #   The year field is only used for forecasts, otherwise ignored
   #   The day field is only used in subseasonal forecasts, otherwise ignored
   #   The month field is an integer representing a month - ie, May=5
  'fdate':  dt.datetime(2023, 2, 1), #dt.datetime(2022, 6, 1),  
    
   # 'first_year':
   #   the first year of hindcasts you want. **NOT ALL MODELS HAVE ALL YEARS**
   #   double check that your model has hindcast data for all years in [first_year, final_year]
   #   This field is defined by a python integer representing a year, ie: 1993
  'first_year': 1991,  
    
   # 'final_year':
   #   the final year of hindcasts you want. **NOT ALL MODELS HAVE ALL YEARS**
   #   double check that your model has hindcast data for all years in [first_year, final_year]
   #   This field is defined by a python integer representing a year, ie: 2016
  'final_year': 2020,  
    
   # 'predictor_extent':
   #   The geographic bounding box of the climate model data you want to download
   #   This field is defined by a python dictionary with the keys "north", "south",
   #   "east", and "west", each of which maps to a python integer representing the 
   #   edge of a bounding box. i.e., "north" will be the northernmost boundary,
   #   "south" the southernmost boundary. Example: {"north": 90, "south": 90, "east": 0, "west": 180}
  'predictor_extent': {
    'east':  -67, #20,
    'west': -78, #-20, 
    'north': -18, #20,
    'south': -57 #0 
  }, 
    
   # 'predictand_extent':
   #   The geographic bounding box of the observation data you want to download
   #   This field is defined by a python dictionary with the keys "north", "south",
   #   "east", and "west", each of which maps to a python integer representing the 
   #   edge of a bounding box. i.e., "north" will be the northernmost boundary,
   #   "south" the southernmost boundary. Example: {"north": 90, "south": 90, "east": 0, "west": 180}
  'predictand_extent': {
    'east':  -67, #10, 
    'west': -78, #-18,  
    'north': -18, #18,  
    'south': -57 #3 
  },

    
   # 'lead_low': 
   #   the number of months from the first of the initialization month to the center of 
   #   the first month included in the target period. Always an integer + 0.5. 
   #   this field is defined by a python floating point number 
   #   for example  a lead-1 forecast would use lead_low=1.5, if you want init=may, target=Jun-..
  'lead_low': 1.5,
    
   # 'lead_high': 
   #   the number of months from the first of the initialization month to the center of 
   #   the last month included in the target period. Always an integer + 0.5. 
   #   this field is defined by a python floating point number 
   #   for example  a forecast initialized in may, whose target period ended in Aug, 
   #   would use lead_high=3.5
  'lead_high': 3.5, 
    
   # 'target': 
   #   Mmm-Mmm indicating the months included in the target period of the forecast. 
   #   this field is defined by a python string, with two three-letter month name abbreviations 
   #   whose first letters are capitalized, and all other letters are lowercase
   #   and who are separated by a dash character. 
   #   for example, if you wanted a JJA target period, you would use 'Jun-Aug'
  'target': 'Mar-May',#'Jul-Sep',
    
   # 'filetype':
   #   the filetype to be downloaded. for now, it saves a lot of headache just to set this equal
   #   to 'cptv10.tsv' which is a boutique plain-text CPT filetype based on .tsv + metadata
  'filetype': 'cptv10.tsv'
}

cpt_args = { 
    'transform_predictand': None,  # transformation to apply to the predictand dataset - None, 'Empirical', 'Gamma'
    'tailoring': None,  # tailoring None, 'Anomaly', 'StdAnomaly', or 'SPI' (SPI only available on Gamma)
    'cca_modes': (1,3), # minimum and maximum of allowed CCA modes 
    'x_eof_modes': (1,8), # minimum and maximum of allowed X Principal Componenets 
    'y_eof_modes': (1,6), # minimum and maximum of allowed Y Principal Components 
    'validation': 'crossvalidation', # the type of validation to use - crossvalidation, retroactive, or doublecrossvalidation
    'drymask': False, #whether or not to use a drymask of -999
    'scree': True, # whether or not to save % explained variance for eof modes
    'crossvalidation_window': 5,  # number of samples to leave out in each cross-validation step 
    'synchronous_predictors': True, # whether or not we are using 'synchronous predictors'
}


force_download = True

In [ ]:
files_root = notebook.setup(download_args, caseDir)
outputDir = files_root / "output"

In [ ]:
# Uncomment the following line & change the config filepath to save this configuration: 
config_file = ce.save_configuration(caseDir+'.config', download_args, cpt_args, MOS, predictor_names, predictand_name )

# Uncomment the following line & change the config filepath to load an existing configuration: 
#MOS, download_args, cpt_args, predictor_names, predictand_name = ce.load_configuration('test1.config')

#### Download Observations, Hindcasts, and Forecasts

In [ ]:
Y, hindcast_data, forecast_data = notebook.download_data(predictand_name, predictor_names, download_args, files_root, force_download)

#### Perform Analysis 

In [ ]:
hcsts, fcsts, skill, pxs, pys = notebook.evaluate_models(hindcast_data, MOS, Y, forecast_data, cpt_args, outputDir, predictor_names)

#### Plot skill 

In [ ]:
notebook.plot_skill(predictor_names, skill, MOS, files_root)

#### Plot CCA Modes

In [ ]:
notebook.plot_cca_modes(MOS, predictor_names, pxs, pys, files_root)

#### Plot EOF Modes

In [ ]:
notebook.plot_eof_modes(MOS, predictor_names, pxs, pys, files_root)

#### Plot Forecasts

In [ ]:
notebook.plot_forecasts(cpt_args, predictand_name, fcsts, files_root, predictor_names, MOS)

# Multi-Model Ensemble

In [ ]:
#ensemble = ['CFSv2.PRCP','SEAS5.PRCP']
ensemble = predictor_names

det_fcst, pr_fcst, pev_fcst, nextgen_skill = notebook.construct_mme(fcsts, hcsts, Y, ensemble, predictor_names, cpt_args, outputDir)

#### Plot MME Forecast Skill

In [ ]:
notebook.plot_mme_skill(predictor_names, nextgen_skill, MOS, files_root)

#### Plot MME Forecasts

In [ ]:
notebook.plot_mme_forecasts(cpt_args, predictand_name, pr_fcst, MOS, files_root, det_fcst)

#### Construct MME Flexible Forecasts

In [ ]:
# if 'isPercentile is True, the threshold is a percentile (e.g., 0.5)
# else in the unit of the predictand (e.g., mm, degC, ...)
threshold = 0.5
isPercentile = True

exceedance_prob, fcst_scale, climo_scale, fcst_mu, climo_mu, Y2, ntrain, transformed_threshold = notebook.construct_flex_fcst(MOS, cpt_args, det_fcst, threshold, isPercentile, Y, pev_fcst)

#### Plot Flexible MME Forecasts

In [ ]:
# Choose a gridpoint within the predictand domain to plot the forecast and climatological
# probability of exceedance and PDF curves. If set to None, the centroid of the predictand
# domain will be used.

point_latitude = None
point_longitude = None

notebook.plot_mme_flex_forecasts(predictand_name, exceedance_prob, point_latitude, point_longitude, download_args["predictand_extent"], transformed_threshold, fcst_scale, climo_scale, fcst_mu, climo_mu, Y2, cpt_args['transform_predictand'], ntrain, Y, MOS, files_root)